In [80]:
import os
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
!pip install autocorrect
from autocorrect import Speller
from collections import OrderedDict 
from pathlib import Path
import shutil
import spacy 
import re
nltk.download('wordnet')
nltk.download('stopwords')
!pip install contractions
!pip install spacy
!pip install textblob
import contractions
from textblob import TextBlob, Word
import pandas as pd
import numpy as np
import ast

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
paths = []
filename = []
# /content/drive/MyDrive/stories
# /content/drive/MyDrive/IR ASSIGNMENT 2/stories
for (dirpath, dirnames, filenames) in os.walk("/content/drive/MyDrive/stories"):
  for i in filenames:
    paths.append(str(dirpath)+str("/")+i)
    filename.append(i)

In [ ]:
len(paths)

467

In [ ]:
documentsDict = {}
# filenameOfDoc = {}
# global counter 
# counter = 0
def cleaning(filepath,filename,flag):
  if(flag==1):
    f = open(filepath, 'r', errors = 'ignore',encoding='cp1250').read().strip()
    clean_S = contractions.fix(f)
    clean_S=re.sub("([\-]{1,2})", " ",clean_S)
    clean_s=re.sub("[\,]", " ",clean_S)
    clean_S = re.sub("[\.]"," ",clean_S)
    clean_S = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", clean_S)  #junk symbols
    # clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S=re.sub("([\.\!]{2})", " ",clean_S)  #removing punctuations
    clean_S = re.sub('\s+', ' ', clean_S)    
    clean_S=re.sub("[^a-zA-Z\s\n]", " ",clean_S)  #used to eliminate non-ascii and numeric symbols 
    clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S = clean_S.lower()
    documentsDict[filename]=clean_S
    # counter = counter+1
    # filenameOfDoc[counter] = filename

  elif(flag==0):
    filepath = contractions.fix(filepath)
    filepath=re.sub("([\-]{1,2})", " ",filepath)
    filepath=re.sub("[0-9]", "",filepath)
    # filepath=re.sub("[\,]", " ",filepath)
    filepath = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", filepath)
    # filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath=re.sub("([\.\!]{2})", " ",filepath)
    filepath = re.sub('\s+', ' ', filepath)
    filepath=re.sub("[^a-zA-Z\s\n]", " ",filepath)
    filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath = filepath.lower()
    return filepath

In [ ]:
for i in range(0,len(filename)):
       cleaning(paths[i],filename[i],1)

In [ ]:
len(documentsDict)

467

In [79]:
f = open('cleaneddidSomeChange.txt', 'w')
f.write(str(documentsDict))
f.close()

In [ ]:
f = open('cleaneddidSomeChange.txt', 'r')
documentsDict = f.read()
f.close()

In [ ]:
def gen_token(flag,query):
  tokenizer = RegexpTokenizer(r'\w+')
  if(flag==1):
    for docname in documentsDict:
        documentsDict[docname] = tokenizer.tokenize(str(documentsDict[docname]))

  elif(flag==0):
    query = tokenizer.tokenize(query)
    return query

In [ ]:
def remove_stopword(flag,query):
  stopwordsList = stopwords.words('english')
  if(flag==1):
    for docname in documentsDict:
        for word in documentsDict[docname]:
            if word in stopwordsList:
                documentsDict[docname].remove(word)
  elif(flag==0):
      q=[]
      for word in query:
            if word not in stopwordsList:
              q.append(word)
      query=q.copy()
      return query

In [ ]:
def normalise_using_textblob(flag,query):
  if(flag==1):       
    for docname in documentsDict:
        dataList = []
        for word in documentsDict[docname]:
            w = Word(word)
            dataList.append(w.lemmatize())
        documentsDict[docname] = dataList   
  elif(flag==0):
    ans = []
    for word in query:
      w=Word(word)
      ans.append(w.lemmatize())
    return ans

In [ ]:
def proquery(query):
  queryFinal=[]
  query = cleaning(query," ",0)
  queryFinal = gen_token(0,query)
  queryFinal = remove_stopword(0,queryFinal)
  queryFinal = normalise_using_textblob(0,queryFinal)
  #queryFinal = queryFinal.replace(","," ").split()
  return queryFinal

In [ ]:
#preprocessing DocumentDict after Cleaning
gen_token(1,"")
remove_stopword(1,"")
normalise_using_textblob(1,"")

In [ ]:
len(documentsDict)

467

In [ ]:
documentsFilename = list(documentsDict.keys())

In [ ]:
documentsFilename
f = open("/content/drive/MyDrive/IR/documentsFilenameLemFinal.txt","w")
f.write(str(documentsFilename))
f.close()

In [ ]:
f = open("/content/drive/MyDrive/IR/documentsFilenameLemFinal.txt","r")
x = f.read()
x = x.replace("'","")
x = x.replace("[","")
x = x.replace("]","")
documentsFilename = x.split(",")
f.close()


#Positional List

In [ ]:
Postings = pd.DataFrame()

Final Postings

In [ ]:
for docid in range(0,len(documentsFilename)):
  tokensOfDocid = documentsDict[documentsFilename[docid]]
  for position in range(0,len(tokensOfDocid)):
    word = tokensOfDocid[position]
    if word in Postings:
      if docid in Postings.loc[1][word]:
        Postings.loc[1][word][docid].append(position)
      else:
        Postings.iloc[1][word][docid] = [position]
        Postings.iloc[0][word] = Postings.iloc[0][word] + 1
    else:
      Postings.insert(value=[1,{docid:[position]}], loc=0, column=word)

In [ ]:
Postings.to_csv("/content/drive/MyDrive/IR/PostingsLemFinal.csv")

#Support for searching

In [ ]:
Postings = pd.read_csv("/content/drive/MyDrive/IR/PostingsLemFinal.csv")

In [ ]:
Postings = Postings.drop(["Unnamed: 0"],axis=1)

In [ ]:
Postings

,suborner,reconcilement,depose,vigilant,idolatrous,inhumanity,distributive,unclique,recalcitrat,yerks,unacquaintedness,cortez,ferdinando,remarkers,detecters,reflecters,considerers,answerer,illwill,descanting,brobdingnagians,finget,improba,mendacemque,etiam,vanum,chparagraphfinxit,sinonem,fortuna,miserum,impudently,credulity,falsity,copulating,rotherhith,accoutred,inquisition,abhorring,mendez,lisbon,...,new,daves,floor,paneled,sitting,snow,outside,focused,eye,asked,left,anything,laying,can,bottle,array,hitting,shifting,leg,replied,yeah,anyway,called,really,ahead,go,mind,look,said,dave,cancer,dropped,jamie,waited,hand,extended,mark,cigarette,another,give
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,250,8,166,2,133,39,156,27,298,220,283,244,44,303,83,24,37,23,138,145,97,122,228,236,95,320,230,302,348,25,11,127,2,121,289,56,82,40,300,245
1,{466: [65626]},{466: [65562]},{466: [65154]},{466: [65055]},{466: [64947]},{466: [64913]},{466: [64772]},{466: [64654]},{466: [64653]},{466: [64638]},{466: [64591]},{466: [64491]},{466: [64490]},{466: [64413]},{466: [64412]},{466: [64411]},{466: [64409]},{466: [64408]},{466: [64306]},{466: [64210]},"{466: [64192, 64526]}",{466: [63996]},{466: [63995]},{466: [63994]},{466: [63993]},{466: [63992]},{466: [63991]},{466: [63989]},{466: [63988]},{466: [63987]},{466: [63900]},{466: [63896]},{466: [63837]},{466: [63398]},{466: [63297]},{466: [62917]},{466: [62880]},{466: [62735]},"{466: [62043, 63372]}","{466: [61887, 62611, 62790, 63195]}",...,"{0: [47], 1: [178, 201, 221, 447], 2: [719, 72...","{0: [46, 84, 132], 32: [64, 147, 317, 896], 14...","{0: [45], 1: [391], 2: [149, 589, 2970, 3564, ...","{0: [44], 32: [60]}","{0: [43, 69, 105, 492], 2: [660, 1057, 1081, 1...","{0: [42, 172, 195, 217, 555], 6: [8906, 8913, ...","{0: [41, 147, 628], 2: [198, 930, 3033], 4: [6...","{0: [40], 32: [55], 72: [330], 96: [1004, 3400...","{0: [39, 144], 2: [1000, 1297, 1551, 1757, 188...","{0: [36, 646, 816], 1: [165], 2: [280, 654, 24...","{0: [33, 203], 1: [190, 325], 2: [699, 1104, 1...","{0: [32, 271, 765], 2: [599, 674, 1910, 3133],...","{0: [31, 167], 17: [423], 32: [46], 64: [202],...","{0: [30, 297, 677], 2: [2819, 3953], 4: [735, ...","{0: [29, 34], 5: [956, 1680, 1699, 1715, 1764,...","{0: [28], 5: [1191], 18: [1342], 23: [1095], 4...","{0: [27], 6: [10256, 18703, 31068], 16: [529],...","{0: [26], 23: [1802], 31: [6211], 32: [41], 86...","{0: [25], 1: [387], 2: [2980, 2991], 3: [42, 7...","{0: [24], 1: [114], 6: [2737, 3580, 8551, 9744...","{0: [22, 220, 236, 370, 425, 647, 778], 2: [75...","{0: [21], 1: [497], 2: [534], 3: [297], 6: [19...","{0: [20], 1: [303, 602], 2: [617, 1452, 2275, ...","{0: [18, 819], 1: [15], 4: [1382], 5: [841, 10...","{0: [17], 2: [2953], 4: [1645, 1717], 6: [1214...","{0: [16, 358, 383, 758], 1: [198], 2: [1055, 2...","{0: [15], 2: [1987, 3429], 3: [414], 5: [2013,...","{0: [14, 698, 815], 2: [356, 438, 975, 1492, 2...","{0: [12, 38, 216, 222, 248, 307, 372, 417, 443...","{0: [11, 35, 66, 235, 276, 280, 318, 371, 442,...","{0: [10, 19], 31: [4518], 32: [24, 33], 66: [4...","{0: [8], 5: [5326], 6: [2291, 2997, 13739, 182...","{0: [7, 62, 116, 139, 215, 249, 299, 416, 484,...","{0: [6], 2: [812, 3224, 3317], 3: [240], 6: [1...","{0: [5], 2: [7, 66, 558, 656, 682, 958, 1170, ...","{0: [4], 6: [178, 4507, 9309, 10478, 18922, 34...","{0: [3, 13, 23, 37, 49, 142, 221, 284, 325, 34...","{0: [2, 9, 56, 798], 6: [5232, 32374, 32417, 3...","{0: [1], 2: [847, 996, 1513, 1706, 3192, 3627]...","{0: [0], 2: [2790], 4: [474, 803, 1443], 5: [2..."


In [ ]:
for column in Postings.columns:
  xa = ast.literal_eval(Postings.iloc[1][column])
  Postings.iloc[1][column] = xa

In [81]:
f = open("/content/drive/MyDrive/IR/documentsFilenameLemFinal.txt","r")
x = f.read()
x = x.replace("'","")
x = x.replace("[","")
x = x.replace("]","")
documentsFilename = x.split(",")
f.close()

In [82]:
query = "GOOD DAYS"
#do preprocessing
tokenquery = proquery(query) #['to', 'be', 'or', 'not']
#get machlist at each bi-iteration
tokenPostingFirst = {}
tempPosting = {}
for token in range(0,len(tokenquery)-1):
  if tokenquery[token] in Postings:
    if tokenPostingFirst == {}:
      tokenPostingFirst = Postings[tokenquery[token]][1]

    else:
      tokenPostingFirst = tempPosting
      tempPosting = {}
    #for nexttoken in range(token,len(tokenquery)):
    if tokenquery[token+1] in Postings: 
      tokenPostingsecond = Postings[tokenquery[token +1]][1]
      #checking same doc or not from tokenpostingfirst and second
      for doc_id in tokenPostingFirst:
        if doc_id in tokenPostingsecond:
          #now checking positions of tokenfirst in docid consequtive or not with tokensecond
          iteration = tokenPostingFirst[doc_id]
          # print(str(type(iteration))+" docid "+str(doc_id)+" token "+str(token))
          # print(iteration)
          for positionOftoken in iteration:
            if positionOftoken+1 in tokenPostingsecond[doc_id]:
              if doc_id in tempPosting:
                tempPosting[doc_id].append(positionOftoken +1)
              else:
                tempPosting[doc_id] = []
                tempPosting[doc_id].append(positionOftoken + 1)
  else:
    break
print("Entered Query : "+ str(tokenquery))
print("Number of Documents Retrieved : "+ str(len(tempPosting.keys())))
print("Documents containing query are following : ")
for docid in tempPosting.keys():
  print(documentsFilename[docid])

Entered Query : ['good', 'day']
Number of Documents Retrieved : 21
Documents containing query are following : 
13chil.txt
 bruce-p.txt
 aesop11.txt
 aesopa10.txt
 fantasy.txt
 history5.txt
 hound-b.txt
 horswolf.txt
 melissa.txt
 mazarin.txt
 sick-kid.txt
 startrek.txt
 srex.txt
 breaks2.asc
 outcast.dos
 brain.damage
 fic5
 forgotte
 superg1
 enchdup.hum
 fantasy.hum


Term Frequency in Docs

In [ ]:
file2 = pd.read_csv("/content/drive/MyDrive/IR ASSIGNMENT 2/PostingsLemFinal.csv")
file2 = file2.drop("Unnamed: 0",axis=1)
for column in file2.columns:
  xa = ast.literal_eval(file2.iloc[1][column])
  file2.iloc[1][column] = xa
for col in file2:
  for doc in file2.iloc[1][col]:
    file2.iloc[1][col][doc]=len(file2.iloc[1][col][doc])
file2

,bordeaux,mademarchj,unlockmarchto,intertwine,elvish,enchant,songmarcha,fleecy,rokraven,staeorra,marsupial,amex,gecko,tissuealice,conjointly,minaret,oriel,saracenic,upspringing,streamlet,intertangled,tuberose,poppy,bosky,laved,begirt,amphitheatre,musically,diverging,wreathe,effulgence,luxuriance,clematis,eglantine,honeysuckle,overspread,cleanness,luxuriantly,divinest,bidden,...,time,long,seen,well,face,frown,slight,replied,yard,across,walked,cried,morning,good,exclaimed,paw,eye,shading,end,walking,himself,said,road,coming,like,look,looking,carrot,big,great,eating,rocking,porch,front,sat,rabbit,mr,fox,sly,child
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,407,302,230,333,276,28,70,145,94,196,194,115,181,297,83,35,298,5,251,109,132,348,121,175,382,302,232,5,214,238,76,11,25,203,197,27,119,24,15,153
1,{466: 1},{466: 1},{466: 1},{466: 1},{466: 1},{466: 1},{466: 1},{466: 1},{466: 1},{466: 1},{465: 1},{465: 1},{465: 1},{464: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},{463: 1},...,"{0: 4, 1: 16, 2: 11, 3: 6, 4: 7, 5: 3, 6: 5, 7...","{0: 4, 1: 20, 2: 11, 3: 5, 4: 2, 11: 4, 12: 3,...","{0: 1, 1: 6, 2: 7, 3: 4, 4: 1, 6: 1, 11: 1, 13...","{0: 3, 1: 14, 2: 38, 3: 39, 5: 2, 6: 5, 8: 1, ...","{0: 2, 1: 16, 2: 12, 3: 8, 5: 2, 6: 1, 8: 1, 1...","{0: 1, 1: 1, 6: 1, 20: 7, 37: 1, 49: 1, 58: 3,...","{0: 2, 3: 1, 14: 1, 15: 1, 19: 1, 20: 28, 23: ...","{0: 4, 4: 1, 6: 2, 8: 1, 9: 2, 10: 1, 11: 6, 1...","{0: 1, 2: 6, 3: 1, 4: 1, 14: 1, 20: 1, 22: 1, ...","{0: 4, 1: 1, 2: 1, 3: 1, 12: 1, 13: 1, 14: 3, ...","{0: 1, 1: 1, 2: 1, 3: 2, 13: 2, 14: 4, 15: 1, ...","{0: 5, 2: 5, 3: 2, 6: 1, 11: 2, 14: 4, 15: 3, ...","{0: 4, 1: 2, 2: 5, 3: 6, 5: 2, 6: 1, 8: 1, 13:...","{0: 6, 1: 16, 2: 11, 3: 13, 4: 1, 5: 2, 9: 1, ...","{0: 2, 6: 2, 8: 2, 11: 2, 17: 2, 18: 1, 19: 2,...","{0: 2, 1: 1, 16: 5, 21: 1, 23: 2, 31: 1, 38: 1...","{0: 3, 1: 29, 2: 6, 3: 4, 5: 1, 6: 3, 9: 2, 10...","{0: 1, 66: 1, 174: 1, 361: 1, 422: 1}","{0: 2, 1: 10, 2: 3, 3: 5, 5: 2, 8: 2, 11: 1, 1...","{0: 2, 13: 1, 14: 2, 18: 1, 20: 12, 21: 3, 34:...","{0: 2, 4: 1, 6: 3, 8: 1, 11: 1, 16: 1, 17: 1, ...","{0: 5, 1: 8, 2: 33, 3: 43, 4: 7, 6: 5, 8: 3, 9...","{0: 2, 1: 9, 2: 11, 5: 9, 12: 3, 13: 1, 14: 3,...","{0: 1, 1: 6, 2: 1, 3: 1, 14: 1, 18: 2, 20: 41,...","{0: 2, 1: 53, 2: 6, 3: 10, 4: 5, 5: 2, 6: 1, 8...","{0: 2, 1: 10, 2: 3, 3: 7, 4: 1, 5: 1, 8: 2, 14...","{0: 1, 1: 1, 2: 2, 3: 2, 12: 2, 14: 5, 15: 2, ...","{0: 1, 66: 1, 74: 1, 86: 3, 344: 2}","{0: 2, 1: 13, 2: 1, 3: 3, 4: 1, 5: 1, 6: 2, 8:...","{0: 2, 1: 6, 2: 7, 3: 5, 4: 1, 5: 3, 6: 2, 7: ...","{0: 1, 1: 1, 5: 1, 13: 1, 16: 1, 21: 3, 26: 1,...","{0: 1, 20: 5, 58: 1, 74: 2, 144: 3, 195: 2, 22...","{0: 3, 21: 2, 45: 1, 74: 3, 96: 2, 102: 1, 128...","{0: 1, 1: 2, 2: 5, 3: 1, 6: 1, 14: 4, 18: 1, 2...","{0: 2, 1: 5, 2: 5, 6: 1, 8: 1, 13: 1, 14: 5, 1...","{0: 33, 5: 1, 18: 1, 54: 1, 58: 1, 60: 1, 65: ...","{0: 27, 2: 39, 3: 32, 10: 2, 14: 56, 15: 37, 1...","{0: 30, 28: 1, 37: 2, 45: 122, 48: 67, 60: 1, ...","{0: 26, 15: 1, 27: 1, 37: 1, 38: 2, 45: 1, 48:...","{0: 1, 1: 9, 5: 1, 6: 3, 11: 2, 14: 4, 16: 3, ..."


In [ ]:
file2.to_csv("/content/drive/MyDrive/IR/FrequencyOfWord2.csv")